In [1]:
import pandas as pd
import numpy as np
import pulp as pl
import geopy.distance as d
from concurrent.futures import ThreadPoolExecutor, as_completed
from collections import defaultdict

In [2]:
shipment = pd.read_csv('Shipments.csv')
location = pd.read_csv('Locations.csv')
bagging = pd.read_csv('Bagging activities.csv')
loc_trans = pd.read_csv('Location transfers.csv')
begin_inv = pd.read_csv('Beginning inventory.csv')

## List of warehouses being considered for the model

In [4]:
grp_frm_df = loc_trans[loc_trans['FISCALYEAR'] == 2024].groupby(['FROMLOCATION']).agg({'NETUNITS' : 'sum'}).reset_index().rename(columns = {'NETUNITS' : 'total_units_from'})
grp_to_df = loc_trans[loc_trans['FISCALYEAR'] == 2024].groupby(['TOLOCATION']).agg({'NETUNITS' : 'sum'}).reset_index().rename(columns = {'NETUNITS' : 'total_units_to'})

grp_df = grp_frm_df.merge(grp_to_df, how = 'outer', left_on = 'FROMLOCATION', right_on = 'TOLOCATION')

grp_df[(grp_df.total_units_to >= 10000) & (grp_df.total_units_from >= 10000)]

,FROMLOCATION,total_units_from,TOLOCATION,total_units_to
1,B,265189.0,B,185336.5
3,E,297559.0,E,270732.0
4,EBF,87424.5,EBF,39320.0
5,FDC,12709.0,FDC,44911.0
6,I,311550.5,I,148700.5
8,M,34252.5,M,56847.5
10,S,48400.0,S,329310.5


In [5]:
# This list was derived from the conditions above
wh_list = ['B', 'E', 'I', 'M', 'S']

## Reference Year

In [7]:
customer_year_of_ref = [2023, 2024]
supply_year_of_ref = 2024

### Calculating all the distances for the required customers and WHs

In [9]:
cust_df = shipment[(shipment['FISCALYEAR'].isin(customer_year_of_ref)) &
                  (shipment['SHIPGPSLATITUDE'] != 0) &
                  (shipment['SHIPGPSLONGITUDE'] != 0)&
                  (shipment['NETUNITS'] <= 0)][['KEYCUSTOMER', 'SHIPGPSLATITUDE', 'SHIPGPSLONGITUDE']].drop_duplicates()

cust_df['latlong'] = cust_df['SHIPGPSLATITUDE'].astype(str) + '-' + cust_df['SHIPGPSLONGITUDE'].astype(str)
cust_df['row_number'] = cust_df.groupby('KEYCUSTOMER')['latlong'].rank('first')
cust_df = cust_df[cust_df['row_number'] == 1]

cust_df.rename(columns = {'KEYCUSTOMER' : 'LOCATION'}, inplace = True)
cust_df['TYPE'] = 'Customer'


wh_df = location[location['LOCATION'].isin(wh_list)][['LOCATION', 'SHIPGPSLATITUDE', 'SHIPGPSLONGITUDE']].drop_duplicates()
wh_df['TYPE'] = 'WH'

cust_wh_df = pd.concat([cust_df, wh_df])
wh_df.rename(columns = {'LOCATION' : 'WH_NAME', 'SHIPGPSLATITUDE' : 'WH_SHIPGPSLATITUDE', 'SHIPGPSLONGITUDE' : 'WH_SHIPGPSLONGITUDE'}, inplace = True)
wh_df = wh_df[['WH_NAME', 'WH_SHIPGPSLATITUDE', 'WH_SHIPGPSLONGITUDE']]
cust_wh_df = wh_df.merge(cust_wh_df, how = 'cross')

cust_wh_df['distance_miles'] = cust_wh_df.apply(
        lambda row: d.geodesic((row['WH_SHIPGPSLATITUDE'], row['WH_SHIPGPSLONGITUDE']), (row['SHIPGPSLATITUDE'], row['SHIPGPSLONGITUDE'])).miles, axis=1
    )


In [10]:
dist_df = cust_wh_df[['WH_NAME', 'LOCATION', 'distance_miles']]
dist_df['distance_miles'] = round(dist_df['distance_miles'], 2)
dist_df = dist_df[(dist_df['distance_miles'] != 0)]

C:\Users\abhis\AppData\Local\Temp\ipykernel_28740\2456627460.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dist_df['distance_miles'] = round(dist_df['distance_miles'], 2)


### Segment Customers

In [12]:
cust_units_df = shipment[(shipment['FISCALYEAR'].isin(customer_year_of_ref)) &
                  (shipment['SHIPGPSLATITUDE'] != 0) &
                  (shipment['SHIPGPSLONGITUDE'] != 0) &
                  (shipment['NETUNITS'] <= 0)][['KEYCUSTOMER', 'NETUNITS']]

cust_units_df = cust_units_df.groupby(['KEYCUSTOMER']).agg({'NETUNITS' : 'sum'}).reset_index()

cust_units_df['NETUNITS'] = 0 - cust_units_df['NETUNITS']


## Modify the thresholds below to change the cluster definition

In [14]:

cust_units_df['cluster'] = np.where(cust_units_df['NETUNITS'] < 800, 'C0',
                                    np.where((cust_units_df['NETUNITS'] >= 800) & (cust_units_df['NETUNITS'] < 2100), 'C2',
                                             np.where(cust_units_df['NETUNITS'] >= 2100, 'C1', 'NA')))


## Defining the variables required for dictionaries for the model

In [16]:
supply_constraint_multiplier = 1.15
high_demand_constraint_mulitplier = 1.1
regular_demand_constraint_mulitplier = 1
low_demand_constraint_mulitplier = 0.9

# defining the dictionary with the required states and their probabilities
scenarios_arg = {'C0-Low Demand, C2-Low Demand, C1-Low Demand' : 0.1,
 'C0-Low Demand, C2-Low Demand, C1-Regular Demand': 0.1,
 'C0-Low Demand, C2-Low Demand, C1-High Demand': 0.1,
 'C0-Low Demand, C2-Regular Demand, C1-Regular Demand': 0.1,
 'C0-Low Demand, C2-Regular Demand, C1-High Demand': 0.1,
 'C0-Low Demand, C2-High Demand, C1-High Demand': 0.1,
 'C0-Regular Demand, C2-Regular Demand, C1-Regular Demand': 0.1,
 'C0-Regular Demand, C2-Regular Demand, C1-High Demand': 0.1,
 'C0-Regular Demand, C2-High Demand, C1-High Demand': 0.1,
 'C0-High Demand, C2-High Demand, C1-High Demand' : 0.1}

warehouses_arg = list(dist_df.WH_NAME.unique())
customers_arg = list(cust_units_df.KEYCUSTOMER.unique())

scenario_list_arg = list(scenarios_arg.keys())


### Converting distance dataframe to a dictionary for the model

In [18]:
%%time

wh_dist_df = dist_df[dist_df.LOCATION.isin(warehouses_arg)]
cust_dist_df = dist_df[~(dist_df.LOCATION.isin(warehouses_arg))]

warehouse_to_customer_distances_arg = {}
warehouse_to_warehouse_distances_arg = {}

for w in warehouses_arg:
  for c in customers_arg:
    # print(c)
    c_dist = cust_dist_df[(cust_dist_df['LOCATION'] == c) & (cust_dist_df['WH_NAME'] == w)]
    warehouse_to_customer_distances_arg[(w, c)] = c_dist['distance_miles'].item()

for w1 in warehouses_arg:
  for w2 in warehouses_arg:
    if w1 != w2:
      w_dist = wh_dist_df[(wh_dist_df['WH_NAME'] == w1) & (wh_dist_df['LOCATION'] == w2)]
      warehouse_to_warehouse_distances_arg[(w1, w2)] = w_dist['distance_miles'].item()
        

CPU times: total: 1min 35s
Wall time: 3min 9s


### Creating warehouse capacity dictionary for the model

In [20]:
# 2. WH Capacity

bag_df = bagging[(bagging['FISCALYEAR'] == supply_year_of_ref) &
                 (bagging['LOCATION'].isin(wh_list)) &
                 (bagging['NETUNITS'] >= 0)
                 ].groupby(['LOCATION', 'PRODSUB0']).agg({'NETUNITS' : 'sum'}).reset_index().rename(columns = {'NETUNITS' : 'NETUNITS_prodn'})

bag_df['NETUNITS_prodn'] = bag_df['NETUNITS_prodn']* supply_constraint_multiplier

begin_inv_df = begin_inv[(begin_inv['FISCALYEAR'] == supply_year_of_ref) &
                 (begin_inv['LOCATION'].isin(wh_list)) &
                 (begin_inv['NETUNITS'] >= 0)
                 ].groupby(['LOCATION', 'PRODSUB0']).agg({'NETUNITS' : 'sum'}).reset_index().rename(columns = {'NETUNITS' : 'NETUNITS_beginbal'})


bag_df = bag_df.merge(begin_inv_df, on = ['LOCATION', 'PRODSUB0'], how = 'outer')
bag_df['NETUNITS_beginbal'] = bag_df['NETUNITS_beginbal'].fillna(0)
bag_df['NETUNITS_prodn'] = bag_df['NETUNITS_prodn'].fillna(0)

bag_df['NETUNITS'] = bag_df['NETUNITS_beginbal'] + bag_df['NETUNITS_prodn']

to_loc_df = loc_trans[(loc_trans['FISCALYEAR'] == supply_year_of_ref) &
                 (loc_trans['TOLOCATION'].isin(wh_list))
                 ].groupby(['TOLOCATION', 'PRODSUB0']).agg({'NETUNITS' : 'sum'}).reset_index().rename(columns = {'TOLOCATION' : 'LOCATION'})

from_loc_df = loc_trans[(loc_trans['FISCALYEAR'] == supply_year_of_ref) &
                 (loc_trans['FROMLOCATION'].isin(wh_list))
                 ].groupby(['FROMLOCATION', 'PRODSUB0']).agg({'NETUNITS' : 'sum'}).reset_index().rename(columns = {'FROMLOCATION' : 'LOCATION'})


cap_df = pd.concat([bag_df,to_loc_df,from_loc_df])

cap_df = cap_df.groupby(['LOCATION', 'PRODSUB0']).agg({'NETUNITS' : 'max'}).reset_index()


### Creating customer-scenario-demand dictionary for the model

In [22]:
%%time

# 3. Customer demand for each scenario - let us assume they are all equal for the time being

scenario_intm_df = cust_units_df[['KEYCUSTOMER', 'cluster']].drop_duplicates()

units_intm_df = shipment[(shipment['FISCALYEAR'] == supply_year_of_ref) &
                  (shipment['SHIPGPSLATITUDE'] != 0) &
                  (shipment['SHIPGPSLONGITUDE'] != 0) &
                  (shipment['NETUNITS'] <= 0)].groupby(['KEYCUSTOMER', 'PRODSUB0']).agg({'NETUNITS' : 'sum'}).reset_index()

units_intm_df['NETUNITS'] = round(0 - units_intm_df['NETUNITS'],2)

scenario_intm_df = scenario_intm_df.merge(units_intm_df, on = ['KEYCUSTOMER'], how = 'inner')

demand_values_df = pd.DataFrame(['High Demand', 'Regular Demand', 'Low Demand'], columns = ['demand'])

scenario_df = scenario_intm_df.merge(demand_values_df, how = 'cross')

scenario_df['NETUNITS'] = np.where(scenario_df['demand'] == 'High Demand', scenario_df['NETUNITS']*high_demand_constraint_mulitplier,
                                np.where(scenario_df['demand'] == 'Low Demand', scenario_df['NETUNITS']*low_demand_constraint_mulitplier, scenario_df['NETUNITS']*regular_demand_constraint_mulitplier))


CPU times: total: 62.5 ms
Wall time: 129 ms


In [23]:

def scenario_df_creator(df, scenario_list):
    df_list = []
    for ky in scenario_list:
        cluster_scenarios = ky.split(',')
        for cs in cluster_scenarios:
            case_df = df[(df['demand'] == cs.split('-')[1].strip()) & (df['cluster'] == cs.split('-')[0].strip())]
            case_df['scenario'] = ky
            df_list.append(case_df)
            
    return df_list

In [24]:
final_scenarios_df_list = scenario_df_creator(scenario_df, scenario_list_arg)
final_scenarios_df = pd.concat(final_scenarios_df_list)

C:\Users\abhis\AppData\Local\Temp\ipykernel_28740\2578629555.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  case_df['scenario'] = ky
C:\Users\abhis\AppData\Local\Temp\ipykernel_28740\2578629555.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  case_df['scenario'] = ky
C:\Users\abhis\AppData\Local\Temp\ipykernel_28740\2578629555.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

# The model

## Defining the variables required for the model

In [27]:

stage1_transfers_penalty = 1.75
truck_capacity_arg = 675

prodsub0_list = list(shipment[(shipment['FISCALYEAR'] == supply_year_of_ref) &
                  (shipment['SHIPGPSLATITUDE'] != 0) &
                  (shipment['SHIPGPSLONGITUDE'] != 0) &
                  (shipment['NETUNITS'] <= 0)].sort_values('NETUNITS', ascending = True)['PRODSUB0'].unique())


In [29]:
%%time
# Initialize the dictionary with defaultdict to handle nested structure
customer_demands_arg = defaultdict(dict)

# Populate with existing data
for _, row in final_scenarios_df.iterrows():
    product = row["PRODSUB0"]
    scenario_customer = (row["scenario"], row["KEYCUSTOMER"])
    customer_demands_arg[product][scenario_customer] = row["NETUNITS"]

# Ensure all products, scenarios, and customers have at least a 0 value
all_products = set(shipment[(shipment['FISCALYEAR'] == 2024) &
                  (shipment['SHIPGPSLATITUDE'] != 0) &
                  (shipment['SHIPGPSLONGITUDE'] != 0) &
                  (shipment['NETUNITS'] <= 0)].sort_values('NETUNITS', ascending = True)['PRODSUB0'].unique())

all_scenarios = set(scenarios_arg.keys())
all_customers = set(cust_units_df.KEYCUSTOMER.unique())

for product in all_products:
    for scenario in all_scenarios:
        for customer in all_customers:
            if (scenario, customer) not in customer_demands_arg[product]:
                customer_demands_arg[product][(scenario, customer)] = 0

# Convert defaultdict to a normal dictionary
customer_demands_arg = dict(customer_demands_arg)

# Initialize the dictionary for warehouse capacities
warehouse_capacities_arg = defaultdict(dict)

# Populate with existing data
for _, row in cap_df.iterrows():
    product = row["PRODSUB0"]
    warehouse = row["LOCATION"]
    warehouse_capacities_arg[product][warehouse] = row["NETUNITS"]

# Ensure all products and warehouses have at least a 0 value
all_warehouses = set(dist_df.WH_NAME.unique())

for product in all_products:
    for warehouse in all_warehouses:
        if warehouse not in warehouse_capacities_arg[product]:
            warehouse_capacities_arg[product][warehouse] = 0

# Convert defaultdict to a normal dictionary
warehouse_capacities_arg = dict(warehouse_capacities_arg)


CPU times: total: 27.2 s
Wall time: 35 s


In [36]:

# Define a function to solve the model for a single product
def solve_optimization(product_id, warehouses, customers, scenarios, warehouse_capacities, warehouse_to_customer_distances, warehouse_to_warehouse_distances, customer_demands, truck_capacity, stage1_transfers_penalty):
    # Retrieve product-specific warehouse capacities and customer demands
    product_warehouse_capacities = warehouse_capacities[product_id]
    product_customer_demands = customer_demands[product_id]
    
    print("Running the model for product " + product_id)
    # === Create Model ===
    model = pl.LpProblem("Ag_Two_Stage_Stochastic_LP", pl.LpMinimize)
    
    # === First Stage Variables: warehouse to customer shipments ===
    x = pl.LpVariable.dicts(
        "x",
        [(w, c, s) for w in warehouses for c in customers for s in scenarios],
        lowBound=0,
        cat='Integer'
    )
    
    # === Second Stage Variables: warehouse to warehouse shipments for each scenario ===
    y = pl.LpVariable.dicts(
        "y",
        [(w1, w2, s) for w1 in warehouses for w2 in warehouses if w1 != w2 for s in scenarios],
        lowBound=0,
        cat='Integer'
    )
    
    # === Objective Function ===
    # Stage 1 cost
    stage1_cost = pl.lpSum(
    scenarios[s] * (x[w, c, s]) * warehouse_to_customer_distances[w, c]
    for w in warehouses for c in customers for s in scenarios
    )
    
    # Stage 2 cost (weighted over scenarios)
    stage2_cost = pl.lpSum(
        scenarios[s] * (y[w1, w2, s]) * warehouse_to_warehouse_distances[w1, w2]
        for w1 in warehouses for w2 in warehouses if w1 != w2 for s in scenarios
    )
    
    # Set total objective
    model += stage1_transfers_penalty*stage1_transfers_penalty*(stage1_cost/truck_capacity) + (stage2_cost/truck_capacity), "Total_Distance_Cost"
    
    # === Constraints ===
    
    # 1. Supply Constraint for each warehouse
    for w in warehouses:
        for s in scenarios:
            # Total outflow (to customers and other warehouses)
            outflow_customers = pl.lpSum(x[w, c, s] for c in customers)
            outflow_warehouses = pl.lpSum(y[w, w2, s] for w2 in warehouses if w2 != w)
            total_outflow = outflow_customers + outflow_warehouses
    
            # Total inflow (from other warehouses)
            inflow_warehouses = pl.lpSum(y[w1, w, s] for w1 in warehouses if w1 != w)
    
            # Supply constraint: outflow ≤ inflow + warehouse capacity
            model += (total_outflow <= inflow_warehouses + product_warehouse_capacities[w]), f"SupplyConstraint_{w}_Scenario_{s}"
    
    # 2. Demand Constraint for each customer and scenario
    for c in customers:
        for s in scenarios:
            total_supply = pl.lpSum(x[w, c, s] for w in warehouses)
            model += (total_supply >= product_customer_demands[s, c]), f"DemandConstraint_{c}_Scenario_{s}"
    
    # === Solve the Model ===
    solver = pl.PULP_CBC_CMD(msg=True)  # Use CBC Solver
    model.solve(solver)
    # print('model done')

    # === Extract Results ===
    results = []

    # Stage 1 Variables: Warehouse to Customer Shipments
    for (w, c, s), var in x.items():
        if var.varValue > 0:  # Only store nonzero values
            results.append(["Warehouse_to_Customer", w, c, s, var.varValue])
    
    # Stage 2 Variables: Warehouse to Warehouse Shipments (Scenario-based)
    for (w1, w2, s), var in y.items():
        if var.varValue > 0:  # Only store nonzero values
            results.append(["Warehouse_to_Warehouse", w1, w2, s, var.varValue])
    
    # Convert results to DataFrame
    results_df = pd.DataFrame(results, columns=["Variable_Type", "Warehouse_1", "Warehouse_2/Customer", "Scenario", "Optimal_Value"])
    
    # Add Objective Function Value and Status
    summary_data = pd.DataFrame([
        ["Objective_Value", "-", "-", "-", pl.value(model.objective)],
        ["Status", "-", "-", "-", pl.LpStatus[model.status]]
    ], columns=["Variable_Type", "Warehouse_1", "Warehouse_2/Customer", "Scenario", "Optimal_Value"])
    
    # Combine results and summary data
    final_results_df = pd.concat([results_df, summary_data], ignore_index=True)
    final_results_df['ProdSub0'] = product_id

    # print('results returned')

    return final_results_df


In [38]:
# Function to handle concurrent execution

def run_parallel_optimization(products, warehouses, customers, scenarios, warehouse_capacities, warehouse_to_customer_distances, warehouse_to_warehouse_distances, customer_demands, truck_capacity, stage1_transfers_penalty, final_df, num_threads=10):
    with ThreadPoolExecutor(max_workers=num_threads) as executor:
        futures = [executor.submit(solve_optimization, prod, warehouses, customers, scenarios, warehouse_capacities, warehouse_to_customer_distances, warehouse_to_warehouse_distances, customer_demands, truck_capacity, stage1_transfers_penalty) for prod in products]
        
        for future in as_completed(futures):
            df_results = future.result()
            if len(df_results) > 0:
                final_df = pd.concat([final_df, df_results])
                
    return final_df

In [40]:
%%time

output_empty_df = pd.DataFrame(columns=["Variable_Type", "Warehouse_1", "Warehouse_2/Customer", "Scenario", "Optimal_Value", "ProdSub0"])

# Run the function
output_df = run_parallel_optimization(prodsub0_list, warehouses_arg, customers_arg, scenarios_arg, warehouse_capacities_arg, warehouse_to_customer_distances_arg, warehouse_to_warehouse_distances_arg, customer_demands_arg, truck_capacity_arg, stage1_transfers_penalty, output_empty_df)


Running the model for product 5618STXRIB
Running the model for product 47C77VT2RIB
Running the model for product 639-03VT2RIB
Running the model for product 5701VT2PRO
Running the model for product 34C14
Running the model for product 6659RR
Running the model for product 6659VT2PRO
Running the model for product ES7514VT2P
Running the model for product 645-16VT2PRO
Running the model for product 645-16VT2RIB
Running the model for product 636-16VT2RIB
Running the model for product 5494VT2RIB
Running the model for product 44C27VT2RIB
Running the model for product 66C28-3110
Running the model for product 643-52VT2RIB
Running the model for product 2475VT2RIB
Running the model for product 643-52VT2PRO
Running the model for product 6499STXRIB
Running the model for product 647-79VT2PRO
Running the model for product 60C12-3330AEZ
Running the model for product 640-12STXRIB
Running the model for product ES7698-3110
Running the model for product ES7514
Running the model for product 51C62VT2RIB
Runnin

In [41]:
output_df.head()

,Variable_Type,Warehouse_1,Warehouse_2/Customer,Scenario,Optimal_Value,ProdSub0
0,Warehouse_to_Customer,B,246981,"C0-Low Demand, C2-Low Demand, C1-Low Demand",288.0,5701VT2PRO
1,Warehouse_to_Customer,B,246981,"C0-Low Demand, C2-Low Demand, C1-Regular Demand",319.0,5701VT2PRO
2,Warehouse_to_Customer,B,246981,"C0-Low Demand, C2-Low Demand, C1-High Demand",351.0,5701VT2PRO
3,Warehouse_to_Customer,B,246981,"C0-Low Demand, C2-Regular Demand, C1-Regular D...",319.0,5701VT2PRO
4,Warehouse_to_Customer,B,246981,"C0-Low Demand, C2-Regular Demand, C1-High Demand",351.0,5701VT2PRO


In [42]:
output_df.to_csv("optimization_results.csv", index=False)